In [1]:
import networkx as nx
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
 
def run_ERRW(G, dw, T, n0=None):
    """
    Running the ERRW.

    Parameters
    ----------
    G : networkx Graph
        Underlying structure.
    dw : float
        Edge-reinforcement.
    T : int
        Number of steps of the walk.
    n0: object (default=None)
        Seed node (has to be in G).
    
    Returns
    -------
    S : list
        List of visited nodes.
    """
    
    def get_next_node(G, initial_node):
        neighbors = list(G.neighbors(initial_node))
        weights = np.array([G[initial_node][neighbor]['weight'] for neighbor in neighbors])
        norm_weights = 1.*weights/sum(weights)
        next_node = np.random.choice(neighbors, p=norm_weights)
        return next_node

    #Setting the weights to the initial value
    nx.set_edge_attributes(G, values=1, name='weight')
    #Setting the seed node
    if n0==None:
        #Random seed
        walker_position = random.choice(list(G.nodes))
    else:
        #Provided seed
        walker_position = n0
    
    #Here I will store the walk sequence
    S = []    
    S.append(walker_position)

    for t in range(T):
        #Choosing the next node proportionally to the weights
        next_node = get_next_node(G, walker_position)
        #Reinforcing the weight of the traversed edge
        G[walker_position][next_node]['weight'] += dw
        walker_position = next_node
        S.append(walker_position)
        
    return S

In [2]:
import os
os.chdir('../')

from tqdm import tqdm

In [ ]:
#SW network parameters
N = int(1e6)
K = 4
p = 0.1

DIR = f'./data/ERRW/SW_k{str(K)}/raw_sequences/'
os.makedirs(DIR, exist_ok = True)

n_sim = 5

#ERRW parameters
# dws = np.logspace(-1,1,21)
dws = np.linspace(0,10,101)
T = int(1e5)

G = nx.connected_watts_strogatz_graph(n=N, k=K, p=p, tries=500)

for dw in tqdm(dws):
    print(dw)
    for sim_id in range(n_sim):
        print(sim_id)

#         G = nx.connected_watts_strogatz_graph(n=N, k=K, p=p, tries=500)
        S = run_ERRW(G, dw, T, n0=None)

        #Saving
        FNAME = 'Seq_SW_p%.3f_dw%.5f_simID%i.txt'%(p, dw, sim_id)
        file = open(DIR+FNAME,'w')
        for s in S:
            file.write("%s\n"%s)
        file.close()